# Import and Dataset

In [1]:
import pandas as pd

In [5]:
df = pd.read_csv(r'Sport car price.csv')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Car Make                 1007 non-null   object
 1   Car Model                1007 non-null   object
 2   Year                     1007 non-null   int64 
 3   Engine Size (L)          997 non-null    object
 4   Horsepower               1007 non-null   object
 5   Torque (lb-ft)           1004 non-null   object
 6   0-60 MPH Time (seconds)  1007 non-null   object
 7   Price (in USD)           1007 non-null   object
dtypes: int64(1), object(7)
memory usage: 63.1+ KB


In [ ]:
# Convert 'odometer' to float after removing commas and extra spaces
df['odometer'] = df['odometer'].replace({',': '', ' ': ''}, regex=True).astype(float)

# Convert 'gallons' to float, removing commas if any
df['gallons'] = df['gallons'].replace({',': ''}, regex=True).astype(float)

# # Apply the function to the columns
# df['cost_per_gallon'] = df['cost_per_gallon'].apply(clean_and_convert)
# df['total_spent'] = df['total_spent'].apply(clean_and_convert)

# # Convert 'mpg' to float, removing commas if any
df['mpg'] = df['mpg'].replace({',': ''}, regex=True).astype(float)

# Convert 'miles' to float, handling missing values and commas
df['miles'] = df['miles'].replace({',': ''}, regex=True).astype(float)
